In [ ]:
!pip install earthengine-api geemap folium


In [4]:
!conda list -n gee_env | grep -E "(earthengine|geemap|pandas|xarray|rasterio)"


earthengine-api           1.6.6                    pypi_0    pypi
geemap                    0.36.2                   pypi_0    pypi
pandas                    2.3.2           py310h0158d43_0    conda-forge
rasterio                  1.4.3           py310hb0078ae_1    conda-forge


In [5]:
print("All required packages are installed.")


All required packages are installed.


In [6]:
"""
Google Earth Engine Visualization Script
Create interactive maps showing CODF features and protected areas
"""

import ee
import geemap
import os
from datetime import datetime

# Initialize Earth Engine
ee.Initialize(project='ee-zcs')

# Load feature collections
CODF_polygons = ee.FeatureCollection("projects/ee-zcs/assets/CODF_polygons")
CODF_lines = ee.FeatureCollection("projects/ee-zcs/assets/CODF_lines")
protected_areas = ee.FeatureCollection('WCMC/WDPA/current/polygons')

# Visualization parameters for protected areas
vis_params = {
    'palette': ['2ed033', '5aff05', '67b9ff', '5844ff', '0a7618', '2c05ff'],
    'min': 0.0,
    'max': 1550000.0,
    'opacity': 0.8,
}

# Create painted image for protected areas
image = ee.Image().float().paint(protected_areas, 'REP_AREA')

# Initialize map centered on study area
Map = geemap.Map(center=[-17.724, 41.104], zoom=6)

# Add layers to map
Map.addLayer(image, vis_params, 'Protected Areas')
Map.addLayer(protected_areas, {}, 'Protected Areas (Inspector)', False)
Map.addLayer(CODF_polygons, {}, 'CODF Polygons')

# Create buffered versions of CODF features
buffer_distance = 25000.0  # 25km buffer

codf_polygons_buff = CODF_polygons.map(lambda f: f.buffer(buffer_distance))
codf_lines_buff = CODF_lines.map(lambda f: f.buffer(buffer_distance))

# Add buffered layers
Map.addLayer(codf_polygons_buff, {}, 'CODF Polygons (25km buffer)')
Map.addLayer(codf_lines_buff, {}, 'CODF Lines (25km buffer)')

# Save map as HTML file
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"gee_visualization_{timestamp}.html"
Map.to_html(filename)

print(f"Map saved as: {filename}")
print("Open this file in your browser to view the interactive map")

# Display the map (for Jupyter environments)
Map


EEException: Please authorize access to your Earth Engine account by running

earthengine authenticate

in your command line, or ee.Authenticate() in Python, and then retry.